### 企查查

### 1.按照红头文件匹配

In [ ]:
import jieba
import pandas as pd

In [ ]:
def get_stop_words():
    with open('data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])

In [ ]:
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']

    
def hadle_official_info():
    '''讲地市的红头文件分词'''
    df = pd.read_csv('./redhead.csv',delimiter='$',header=-1)
    df.head()
    df[1] = df[1].apply(lambda x:strQ2B(x))
    jieba.set_dictionary('data/dict.txt.small')
    x_cut = [] #分词后,去除停用词的语料库

    # 获取停用词列表
    stopwords = get_stop_words()
    cut_res = df[1].apply(cut)
    df[1] = cut_res
    return df

official_info_df = hadle_official_info()

In [ ]:
official_info_df

In [ ]:
handle_desc("我爱你中国,哈哈,呵呵,中国")

In [ ]:
official_info_df.to_csv('redhead2.csv',index=0)

In [ ]:
official_info_df = pd.read_csv('redhead2.csv',index_col=0)
official_info_df.index
official_info_df.loc['青岛','1']
type(official_info_df.loc['青岛','1'])

In [ ]:
def judge_by_official(desc,location):
    '''匹配红头文件, 存在则判定为新动能
       -1 : 未找到红头文件
       0: 匹配失败
       1: 新动能匹配成功
    '''
    try:
        print(location)
        official_info_df.loc[location]
    except KeyError: 
        return -1
    official_info = official_info_df.loc[location,'1']
    official_info = official_info.split('/')
    crossinfo = [w for w in desc if w in official_info]
    print('pattern ',location, ' ',len(crossinfo),'字符')
    if len(crossinfo)>1:
        return 1
    else:
        return 0

desc = '青岛达能环保设备股份有限公司，成立于2006年10月，注册资金6300万元，占地面积15万平方米，生产建筑面积5.5万平方米，制造设备齐全，生产装备达到国内同行业领先水平，目前已具备年产值10亿元人民币的生产能力。青岛达能环保设备股份有限公司属锅炉辅机、环保设备制造行业。主要从事电站锅炉干式除渣系统产品、湿式除渣系统产品及烟气余热利用系统产品的研发、生产与销售。青岛达能环保设备股份有限公司为集研发、生产、销售为一体的国家高新技术企业、国家AAA级信用企业、山东省战略新兴产业重点企业、青岛企业技术中心、青岛民营科技创新先进单位。获得国家民营科技发展贡献奖、国家重点新产品、山东优秀节能成果奖等项目的荣誉称号和奖励。随着公司新业务的扩展，现面向社会诚招以下人才'
desc = handle_desc(desc)
location = '青岛'
judge_by_official(desc,location)

### 2. LDA匹配

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
qichacha_cut = pd.read_csv('qichacha_cut.csv',index_col=0)
x_cut = qichacha_cut['经营范围'].apply(lambda x:x.split(',')).values
print(x_cut)
qichacha_cut.head()

In [ ]:
pos_df = pd.read_excel("pos.xlsx")
pos_desc = pos_df["经营范围"].dropna()
pos_desc = pos_desc.apply(cut)
pos_desc = pos_desc.apply(lambda x:x.split("/"))
pos_desc = pos_desc.values
print(pos_desc.shape)
print(x_cut.shape)
y = np.concatenate((np.ones(pos_desc.shape[0]),np.zeros(x_cut.shape[0])),axis=0)
x_cut = np.concatenate((pos_desc,x_cut),axis=0)
x_cut.shape,y.shape

In [ ]:
def getLDA(topic_num):
    from gensim.models import LdaMulticore
    from gensim.corpora import Dictionary
    # gensim
    dictionary = Dictionary(x_cut)
    print(dictionary)
    corpus = [dictionary.doc2bow(doc) for doc in x_cut]
    lda = LdaMulticore(corpus=corpus,  # LDA训练语料
                   id2word=dictionary, # id到单词的映射表
                   num_topics=topic_num)      # LDA主题数量 
    return lda,corpus,dictionary
topic_num = 5
lda,corpus,dictionary = getLDA(topic_num)
lda.print_topics()

In [ ]:
def get_topic_distribute(lda,doc):
    return lda[doc]

def get_all_doc_topic():
    lda_matrix = np.zeros((x_cut.shape[0],topic_num),dtype='float64')
    for index,doc in enumerate(corpus):
        topics = get_topic_distribute(lda,corpus[index])
        for topic_tuple in topics:
            lda_matrix[index,topic_tuple[0]] = topic_tuple[1]
    return lda_matrix

print(get_topic_distribute(lda,corpus[0]))
print(len(corpus))
# 所有样本主题分布
all_topic = get_all_doc_topic()
# x_neg = all_topic[:1150]
# x_pos = all_topic[1150:]
# # svm正例
# y_neg = np.zeros(x_neg.shape[0])

In [ ]:
# SVM训练
def train_svm(lda_matrix,label):
    clf = SVC()
    clf.fit(lda_matrix, label) 
    return clf

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(all_topic,y,test_size=0.1)
clf = train_svm(x_train,y_train)
clf.score(x_test,y_test)

In [ ]:
import pickle
#保存词典
with open('dict_save', 'wb') as f:                     
    picklestring = pickle.dump(dictionary, f)
#保存lda模型
lda.save('lda.model')
#保存svm模型
with open('svm.pickle', 'wb') as fw:
    pickle.dump(clf, fw)

### 预测

In [5]:
import pickle
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import jieba

with open('/root/DeepLearning/dict_save', 'rb') as f:
    load_dictionary = pickle.load(f)   # read file and build object
from gensim import models
load_lda = models.ldamodel.LdaModel.load('lda.model')
# 加载svm.pickle
with open('/root/DeepLearning/svm.pickle', 'rb') as fr:
    load_clf = pickle.load(fr)


def get_stop_words():
    with open('/root/DeepLearning/data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']
# def predict(dictionary,description,clf):
#     a = dictionary.doc2bow(handle_desc(description))
#     topic_distribute = lda[a]
#     res = clf.predict(topic_distribute)
#     return res


# print(x_cut[0])
# print(y[0])
topic_num = 5
a = load_dictionary.doc2bow(handle_desc("航空航天"))
topic_distribute = load_lda[a]
print(topic_distribute)
topics = np.zeros((1,topic_num))
print(topics)
for tupl in topic_distribute:
    print(tupl)
    topics[0,tupl[0]]=tupl[1]
print(topics)
res = load_clf.predict(topics)
res[0]

[(0, 0.2), (1, 0.2), (2, 0.2), (3, 0.2), (4, 0.2)]
[[0. 0. 0. 0. 0.]]
(0, 0.2)
(1, 0.2)
(2, 0.2)
(3, 0.2)
(4, 0.2)
[[0.2 0.2 0.2 0.2 0.2]]


0.0

### 补全地市

In [ ]:
def get_area():
    df2 = pd.read_csv('qichacha.csv',index_col=0)
    return df2['登记机关'].apply(lambda x:x[:2])
    
qichacha_cut['所在区域']=get_area()
qichacha_cut.head()

In [ ]:
qichacha_cut.shape[0]

In [ ]:
SCRIPT_STR('
import pickle
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import jieba

with open('/root/DeepLearning/dict_save', 'rb') as f:
    load_dictionary = pickle.load(f)   # read file and build object
from gensim import models
load_lda = models.ldamodel.LdaModel.load('lda.model')
# 加载svm.pickle
with open('/root/DeepLearning/svm.pickle', 'rb') as fr:
    load_clf = pickle.load(fr)


def get_stop_words():
    with open('/root/DeepLearning/data/stop_words_utf8.txt') as f:
        content = f.readlines()
        stopwords = [w.strip() for w in content]
        return stopwords

# 全角转换成半角
def strQ2B(ustring):
    '''全角转半角
    ustring : 需要转换的字符串
    '''
    ss = ''
    for s in ustring:
        rstring = ""
        for uchar in s:
            inside_code = ord(uchar)
            if inside_code == 12288:  # 全角空格直接转换
                inside_code = 32
            elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
                inside_code -= 65248
            rstring += chr(inside_code)
        ss = ss + rstring
    return ss
# 对corpus中每个文章分词后滤出停用词
def cut(doc):
    content = list(jieba.cut(doc))
    stopwords = get_stop_words()
    return '/'.join([w for w in content if w not in stopwords and w!=' '])
def handle_desc(desc):
    desc = strQ2B(desc)
    cut = jieba.cut(desc)
    stopwords = get_stop_words()
    return [w for w in cut if w not in stopwords and w!=' ']
# def predict(dictionary,description,clf):
#     a = dictionary.doc2bow(handle_desc(description))
#     topic_distribute = lda[a]
#     res = clf.predict(topic_distribute)
#     return res


# print(x_cut[0])
# print(y[0])
topic_num = 5
a = load_dictionary.doc2bow(handle_desc("航空航天"))
topic_distribute = load_lda[a]
print(topic_distribute)
topics = np.zeros((1,topic_num))
print(topics)
for tupl in topic_distribute:
    print(tupl)
    topics[0,tupl[0]]=tupl[1]
print(topics)
res = load_clf.predict(topics)
res[0]
','')